문제 1  10점  
입력 데이터: 다운로드: https://github.com/hyerica-bdml/class-2020-datascience/tree/master/Week%208/input.zip  
설명 - 영화DB사이트 (MovieLens)에서 수집된 데이터로서 u.data에는 사용자가 영화에 매긴 평점이 담겨있고 u.item에는 영화마다 제목, 카테고리 등의 정보가 담겨 있다.  
입력파일 1 - u.data  
라인마다 사용자의 영화평점이 기록되어 있으며 4개의 속성 "user id \t item id \t rating \t timestamp"가 탭으로 구분되어 작성되어 있다. (즉, split("\t") 로 자를 수 있음)  
모두 integer값이다.  
한 사람이 여러개의 영화에 평점을 매길 수 있으므로 같은 user id로 시작하는 라인이 복수번 나타날수 있다.  
item id는 아래 u.item파일에 있는 영화 아이디 (movie id)를 가리킨다 (즉, foreign key)   
rating은 1에서 5까지 범위이다.  

=> Data frame으로 읽는 방법:
from pyspark.sql.types import *  
udataschema = StructType([  
  StructField("uid", LongType(), False),  
  StructField("mid", LongType(), False),  
  StructField("rate",LongType(), False),  
  StructField("timestamp", LongType(), False)  
])
udata = spark.read.format("csv").option("sep", "\t").schema(udataschema).load("u.data")  

입력파일 2 - u.item  
라인마다 영화 정보가 기록되어 있으며 "movie id | movie title | release date | video release date |  
IMDb URL | unknown | Action | Adventure | Animation |  
Children's | Comedy | Crime | Documentary | Drama | Fantasy |  
Film-Noir | Horror | Musical | Mystery | Romance | Sci-Fi |  
Thriller | War | Western" 의 속성 들이 파이프라인(|)으로 구분되어 작성되어 있다.  
그러나 아래 문제를 보면 맨 앞의 두개 속성, movie id와 movie title 만 사용한다.  
movie id는 이 파일 안에서 한번만 나타난다 (즉, primary key)  

=> Data frame으로 읽는 방법:  
uitemschema = StructType(\[  
  StructField("mid", LongType(), False),  
  StructField("title",StringType(), False)  
])  
uitem = spark.read.format("csv").option("sep", "|").schema(uitemschema).load("u.item")  

문제.
5점을 받은 영화 중에서 알파벳 'T'로 시작하는 영화의 개수?

'T'는 대문자  
주의: 5점을 받은 영화중 알파벳 'T'로 시작하는 영화를 중복해서 세지 않도록!  
제한조건: 위 코드를 이용해 u.data와 u.item을 DataFrame으로 읽어들여 select및 join을 이용하여 계산하시오.  
제한조건: 마지막 action으로 답이 출력되도록 코드를 작성하시오. 즉, 최종 답을 계산하기 위해 pyspark action이 아닌 python 코드를 사용하지 마시오.  
힌트: filter를 수행 시 filter조건을 표현할 때 어떤 column의 부분문자열을 뽑기 위해 pyspark.sql.functions.substring를 사용할 수 있음.   
    예를 들어 uitem이란 data frame의 title이란 column의 첫 글자는 substring(uitem\["title"], 0, 1)으로 지정할 수 있음.  

    
문제 2 10점  
문제.  
사용자들에게 1점도 받고 5점도 받은 적이 있는 영화의 제목의 길이 중 가장 긴 영화제목의 길이?

동일 영화가 어떤 사용자에게는 1점, 어떤 사용자에게는 5점을 받은 경우를 말함  
힌트: 어떤 column의 길이의 max 를 select할 때 max 함수와 length함수를 사용.  
예를 들어 uitem\["title"]의 길이의 max는 select(max(length(uitem\["title"]))) 로 계산 할 수 있음  
제한조건: 마지막 action으로 답이 출력되도록 코드를 작성하시오. 즉, 최종 답을 계산하기 위해 pyspark action이 아닌 python 코드를 사용하지 마시오.  


In [1]:
import findspark
findspark.init()

# create spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("my app").master("local").getOrCreate() # spark deamon 생성 # 이전에는 spark.sparkContext

# get context from the session
sc = spark.sparkContext

In [5]:
from pyspark.sql.types import *
schema = StructType([
    StructField("uid", LongType(), False),
    StructField("mid", LongType(), False),
    StructField("rate", LongType(), False),
    StructField("timestamp", LongType(), False)
])
udata = spark.read.format("csv").option("sep", "\t" ).schema(schema).load("u.data")
print(udata.show(3))
udata.printSchema()

+---+---+----+---------+
|uid|mid|rate|timestamp|
+---+---+----+---------+
|196|242|   3|881250949|
|186|302|   3|891717742|
| 22|377|   1|878887116|
+---+---+----+---------+
only showing top 3 rows

None
root
 |-- uid: long (nullable = true)
 |-- mid: long (nullable = true)
 |-- rate: long (nullable = true)
 |-- timestamp: long (nullable = true)



In [6]:
schema = StructType([
    StructField("mid", LongType(), False),
    StructField("title", StringType(), False)
])
uitem = spark.read.format("csv").option("sep", "|").schema(schema).load("u.item")
print(uitem.show(3))
uitem.printSchema() 

+---+-----------------+
|mid|            title|
+---+-----------------+
|  1| Toy Story (1995)|
|  2| GoldenEye (1995)|
|  3|Four Rooms (1995)|
+---+-----------------+
only showing top 3 rows

None
root
 |-- mid: long (nullable = true)
 |-- title: string (nullable = true)



In [7]:
# join
uitem.join(udata, uitem["mid"] == udata["mid"], "inner").show()

+----+--------------------+---+----+----+---------+
| mid|               title|uid| mid|rate|timestamp|
+----+--------------------+---+----+----+---------+
| 242|        Kolya (1996)|196| 242|   3|881250949|
| 302|L.A. Confidential...|186| 302|   3|891717742|
| 377| Heavyweights (1994)| 22| 377|   1|878887116|
|  51|Legends of the Fa...|244|  51|   2|880606923|
| 346| Jackie Brown (1997)|166| 346|   1|886397596|
| 474|Dr. Strangelove o...|298| 474|   4|884182806|
| 265|Hunt for Red Octo...|115| 265|   2|881171488|
| 465|Jungle Book, The ...|253| 465|   5|891628467|
| 451|       Grease (1978)|305| 451|   3|886324817|
|  86|Remains of the Da...|  6|  86|   3|883603013|
| 257| Men in Black (1997)| 62| 257|   2|879372434|
|1014|Romy and Michele'...|286|1014|   5|879781125|
| 222|Star Trek: First ...|200| 222|   5|876042340|
|  40|To Wong Foo, Than...|210|  40|   3|891035994|
|  29|Batman Forever (1...|224|  29|   3|888104457|
| 785|     Only You (1994)|303| 785|   3|879485318|
| 387|Age of

In [19]:
from pyspark.sql.functions import *
uitem.join(udata, uitem["mid"] == udata["mid"], "inner").filter( col('rate') == 5  ).show()

+----+--------------------+---+----+----+---------+
| mid|               title|uid| mid|rate|timestamp|
+----+--------------------+---+----+----+---------+
| 465|Jungle Book, The ...|253| 465|   5|891628467|
|1014|Romy and Michele'...|286|1014|   5|879781125|
| 222|Star Trek: First ...|200| 222|   5|876042340|
| 387|Age of Innocence,...|122| 387|   5|879270459|
|  95|      Aladdin (1992)| 38|  95|   5|892430094|
| 234|         Jaws (1975)|160| 234|   5|876861185|
| 603|  Rear Window (1954)|278| 603|   5|891295330|
| 327|     Cop Land (1997)|287| 327|   5|875333916|
| 201| Evil Dead II (1987)|246| 201|   5|884921594|
|1137|Beautiful Thing (...|242|1137|   5|879741196|
| 241|Last of the Mohic...|249| 241|   5|879641194|
|   4|   Get Shorty (1995)| 99|   4|   5|886519097|
| 181|Return of the Jed...| 25| 181|   5|885853415|
| 196|Dead Poets Societ...| 59| 196|   5|888205088|
| 143|Sound of Music, T...|290| 143|   5|880474293|
| 423|E.T. the Extra-Te...| 42| 423|   5|881107687|
|  26|Brothe

In [22]:
from pyspark.sql.functions import *
uitem.join(udata, uitem["mid"] == udata["mid"], "inner").filter( col('title')[0:1] == 'T').show()

+----+--------------------+---+----+----+---------+
| mid|               title|uid| mid|rate|timestamp|
+----+--------------------+---+----+----+---------+
|  40|To Wong Foo, Than...|210|  40|   3|891035994|
| 118|      Twister (1996)|291| 118|   2|874833878|
|   1|    Toy Story (1995)|308|   1|   4|887736532|
| 979|Trigger Effect, T...|201| 979|   2|884114233|
| 564|Tales from the Ho...|276| 564|   3|874791805|
| 427|To Kill a Mocking...| 60| 427|   5|883326620|
|1444|That Darn Cat! (1...|254|1444|   3|886475558|
| 111|Truth About Cats ...| 11| 111|   4|891903862|
| 209|This Is Spinal Ta...| 28| 209|   4|881961214|
|  23|  Taxi Driver (1976)|135|  23|   4|879857765|
| 118|      Twister (1996)|265| 118|   4|875320714|
|  96|Terminator 2: Jud...| 42|  96|   5|881107178|
| 195|Terminator, The (...| 44| 195|   5|878347874|
| 385|    True Lies (1994)|194| 385|   2|879524643|
| 195|Terminator, The (...| 72| 195|   5|880037702|
|  23|  Taxi Driver (1976)| 59|  23|   5|888205300|
| 379|Tales 

In [27]:
from pyspark.sql.functions import *
uitem.join(udata, uitem["mid"] == udata["mid"], "inner").filter( col('title')[0:3] == 'Ter').show()

+---+--------------------+---+---+----+---------+
|mid|               title|uid|mid|rate|timestamp|
+---+--------------------+---+---+----+---------+
| 96|Terminator 2: Jud...| 42| 96|   5|881107178|
|195|Terminator, The (...| 44|195|   5|878347874|
|195|Terminator, The (...| 72|195|   5|880037702|
| 96|Terminator 2: Jud...|193| 96|   1|889124507|
| 96|Terminator 2: Jud...|200| 96|   5|884129409|
| 96|Terminator 2: Jud...|197| 96|   5|891409839|
| 96|Terminator 2: Jud...|263| 96|   4|891298336|
|586|Terminal Velocity...|279|586|   4|892864663|
| 96|Terminator 2: Jud...|239| 96|   5|889178798|
| 96|Terminator 2: Jud...|188| 96|   5|875073128|
| 96|Terminator 2: Jud...|279| 96|   4|875310606|
|195|Terminator, The (...| 13|195|   3|881515296|
|195|Terminator, The (...| 42|195|   5|881107949|
|195|Terminator, The (...| 77|195|   5|884733695|
| 96|Terminator 2: Jud...|313| 96|   5|891015144|
| 96|Terminator 2: Jud...|262| 96|   4|879793022|
| 96|Terminator 2: Jud...|175| 96|   3|877108051|


In [29]:
from pyspark.sql.functions import *
uitem.join(udata, uitem["mid"] == udata["mid"], "inner").filter( substring(uitem["title"], 0, 1) == 'T').show()

+----+--------------------+---+----+----+---------+
| mid|               title|uid| mid|rate|timestamp|
+----+--------------------+---+----+----+---------+
|  40|To Wong Foo, Than...|210|  40|   3|891035994|
| 118|      Twister (1996)|291| 118|   2|874833878|
|   1|    Toy Story (1995)|308|   1|   4|887736532|
| 979|Trigger Effect, T...|201| 979|   2|884114233|
| 564|Tales from the Ho...|276| 564|   3|874791805|
| 427|To Kill a Mocking...| 60| 427|   5|883326620|
|1444|That Darn Cat! (1...|254|1444|   3|886475558|
| 111|Truth About Cats ...| 11| 111|   4|891903862|
| 209|This Is Spinal Ta...| 28| 209|   4|881961214|
|  23|  Taxi Driver (1976)|135|  23|   4|879857765|
| 118|      Twister (1996)|265| 118|   4|875320714|
|  96|Terminator 2: Jud...| 42|  96|   5|881107178|
| 195|Terminator, The (...| 44| 195|   5|878347874|
| 385|    True Lies (1994)|194| 385|   2|879524643|
| 195|Terminator, The (...| 72| 195|   5|880037702|
|  23|  Taxi Driver (1976)| 59|  23|   5|888205300|
| 379|Tales 

In [24]:
uitem.join(udata, uitem["mid"] == udata["mid"], "inner").filter( (col('rate') == 5) & (col('title')[0:1] == 'T') ).show()

+---+--------------------+---+---+----+---------+
|mid|               title|uid|mid|rate|timestamp|
+---+--------------------+---+---+----+---------+
|427|To Kill a Mocking...| 60|427|   5|883326620|
| 96|Terminator 2: Jud...| 42| 96|   5|881107178|
|195|Terminator, The (...| 44|195|   5|878347874|
|195|Terminator, The (...| 72|195|   5|880037702|
| 23|  Taxi Driver (1976)| 59| 23|   5|888205300|
|379|Tales From the Cr...|286|379|   5|877533771|
|427|To Kill a Mocking...|305|427|   5|886323090|
|209|This Is Spinal Ta...|218|209|   5|877488546|
| 96|Terminator 2: Jud...|200| 96|   5|884129409|
| 96|Terminator 2: Jud...|197| 96|   5|891409839|
|715|   To Die For (1995)| 59|715|   5|888205921|
| 60|Three Colors: Blu...| 60| 60|   5|883327734|
|  1|    Toy Story (1995)|287|  1|   5|875334088|
|845|That Thing You Do...|274|845|   5|878945579|
| 92| True Romance (1993)| 59| 92|   5|888204997|
|161|      Top Gun (1986)|303|161|   5|879468547|
|937|Thousand Acres, A...|174|937|   5|886432989|


In [25]:
uitem.join(udata, uitem["mid"] == udata["mid"], "inner").filter( (col('rate') == 5) & (col('title')[0:1] == 'T') ).distinct().show()

+---+--------------------+---+---+----+---------+
|mid|               title|uid|mid|rate|timestamp|
+---+--------------------+---+---+----+---------+
|282|Time to Kill, A (...| 57|282|   5|883697223|
|  1|    Toy Story (1995)|291|  1|   5|874834481|
| 59|Three Colors: Red...|527| 59|   5|879455792|
| 59|Three Colors: Red...|354| 59|   5|891218208|
|195|Terminator, The (...|495|195|   5|888633396|
|  1|    Toy Story (1995)|523|  1|   5|883701763|
| 23|  Taxi Driver (1976)|615| 23|   5|879448547|
|  1|    Toy Story (1995)|893|  1|   5|874827725|
|475|Trainspotting (1996)|232|475|   5|880062469|
|385|    True Lies (1994)| 42|385|   5|881108147|
|513|Third Man, The (1...|138|513|   5|879024043|
|111|Truth About Cats ...|524|111|   5|884323426|
|  7|Twelve Monkeys (1...|758|  7|   5|881975243|
|195|Terminator, The (...|867|195|   5|880078452|
|513|Third Man, The (1...|387|513|   5|886483330|
|427|To Kill a Mocking...|903|427|   5|891466376|
|845|That Thing You Do...|274|845|   5|878945579|


In [38]:
uitem.join(udata, uitem["mid"] == udata["mid"], "inner").filter( (col('rate') == 5) & (col('title')[0:1] == 'T') ).distinct().count()

1507

### 정리

In [41]:
uitem.select("mid", "title")\
.join(udata.select("mid", "rate"), uitem["mid"] == udata["mid"], "inner")\
.filter( (col('rate') == 5) & (col('title')[0:1] == 'T') ).show()

+---+--------------------+---+----+
|mid|               title|mid|rate|
+---+--------------------+---+----+
|427|To Kill a Mocking...|427|   5|
| 96|Terminator 2: Jud...| 96|   5|
|195|Terminator, The (...|195|   5|
|195|Terminator, The (...|195|   5|
| 23|  Taxi Driver (1976)| 23|   5|
|379|Tales From the Cr...|379|   5|
|427|To Kill a Mocking...|427|   5|
|209|This Is Spinal Ta...|209|   5|
| 96|Terminator 2: Jud...| 96|   5|
| 96|Terminator 2: Jud...| 96|   5|
|715|   To Die For (1995)|715|   5|
| 60|Three Colors: Blu...| 60|   5|
|  1|    Toy Story (1995)|  1|   5|
|845|That Thing You Do...|845|   5|
| 92| True Romance (1993)| 92|   5|
|161|      Top Gun (1986)|161|   5|
|937|Thousand Acres, A...|937|   5|
|513|Third Man, The (1...|513|   5|
|644|Thin Blue Line, T...|644|   5|
| 96|Terminator 2: Jud...| 96|   5|
+---+--------------------+---+----+
only showing top 20 rows



In [42]:
uitem.select("mid", "title")\
.join(udata.select("mid", "rate"), uitem["mid"] == udata["mid"], "inner")\
.filter( (col('rate') == 5) & (col('title')[0:1] == 'T') )\
.distinct().show()

+----+--------------------+----+----+
| mid|               title| mid|rate|
+----+--------------------+----+----+
| 111|Truth About Cats ...| 111|   5|
| 313|      Titanic (1997)| 313|   5|
| 712|      Tin Men (1987)| 712|   5|
| 790|    Tommy Boy (1995)| 790|   5|
|  61|Three Colors: Whi...|  61|   5|
| 592|   True Crime (1995)| 592|   5|
| 797|      Timecop (1994)| 797|   5|
|1122|They Made Me a Cr...|1122|   5|
| 379|Tales From the Cr...| 379|   5|
|1358|The Deadly Cure (...|1358|   5|
|1203|      Top Hat (1935)|1203|   5|
|   1|    Toy Story (1995)|   1|   5|
| 195|Terminator, The (...| 195|   5|
|1260|Total Eclipse (1995)|1260|   5|
| 284|      Tin Cup (1996)| 284|   5|
| 715|   To Die For (1995)| 715|   5|
| 513|Third Man, The (1...| 513|   5|
| 426|Transformers: The...| 426|   5|
|  23|  Taxi Driver (1976)|  23|   5|
|  96|Terminator 2: Jud...|  96|   5|
+----+--------------------+----+----+
only showing top 20 rows



In [44]:
uitem.select("mid", "title")\
.join(udata.select("mid", "rate"), uitem["mid"] == udata["mid"], "inner")\
.filter( (col('rate') == 5) & (col('title')[0:1] == 'T') )\
.distinct().count()

75